# Introduction to NLP with tensorflow

In [ ]:
# import the helperfunction file
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-11-30 18:50:34--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-11-30 18:50:34 (74.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import plot_loss_curves, unzip_data, create_tensorboard_callback, compare_historys

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-11-30 18:50:37--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.69.128, 173.194.197.128, 64.233.191.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2021-11-30 18:50:37 (69.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
unzip_data('nlp_getting_started.zip')

In [ ]:
# IMPORTS
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [ ]:
train_dir = 'train.csv'
test_dir = 'test.csv'

In [ ]:
df = pd.read_csv(train_dir,sep=',')

In [ ]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
df.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [ ]:
df.text[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [ ]:
# get a random row to check the data and label
row = random.randint(0,len(df)-1)
print(f'the row {df.iloc[row].text} has label {df.iloc[row].target}')

the row Bairstow dropped his buffet ticket there. Devastated for the lad. has label 0


In [ ]:
# shuffle the training data
train_df_shuffled = df.sample(frac=1,random_state=711)

In [ ]:
train_df_shuffled.head()

,id,keyword,location,text,target
3424,4897,explode,"Cleveland, TN",VINE OF THE YEAR OH MY GOD I AM ABOUT TO EXPLO...,0
665,960,blaze,Durham N.C,@GuiltyGearXXACP yeah I know but blaze blue do...,0
7533,10771,wreckage,Mumbai,Wreckage 'Conclusively Confirmed' as From MH37...,1
6467,9251,sunk,NaN,descended or sunk however it may be to the sha...,0
6640,9508,terrorist,Iraq|Afghanistan| RSA |Baghdad,Seek help warra #MetroFmTalk,0


In [ ]:
# what does the test df look like
test_df = pd.read_csv(test_dir,sep=',')

In [ ]:
# look the same like the train df but without labels
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# how many examples of each class are there
train_df_shuffled.target.value_counts()
# we have a slightly imbalanced dataset but this is still ok

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# how many samples
len(train_df_shuffled), len(test_df)

(7613, 3263)

In [ ]:
# lets visualize some random data examples
random_indexes = [random.randint(0,len(train_df_shuffled)-1) for _ in range(10)]
for row in df.iloc[random_indexes].itertuples():
  print('-'*20)
  print(f'{row.target}' + (' means not a real disaster' if row.target == 0 else ' means disaster'))
  print(row.text + '\n')


--------------------
0 means not a real disaster
Casualty insurance jobs against hunt up willinghearted into: RPN http://t.co/pByA7Uv3V5

--------------------
0 means not a real disaster
Fire waves and darkness

--------------------
1 means disaster
24 killed in two simultaneous rail crash as acute floods derail the two trains #India #mumbai... http://t.co/4KBWPCmMbM

--------------------
1 means disaster
Ashley and I on going to hurricane harbor Friday. ?? http://t.co/ScEfPFvAEU

--------------------
0 means not a real disaster
@MrTophyPup it's bloody sexy *drools*

--------------------
1 means disaster
Does anyone know why #murfreesboro #walmart was evacuated this evening? @dnj_com

--------------------
0 means not a real disaster
@david_hurn @ToKTeacher Why is there something in place to prevent skynet? Because perhaps there should be! http://t.co/73Umw2iGRZ

--------------------
1 means disaster
#science Now that a piece of wreckage from flight MH370 has been confirmed on RÌ©union 

In [ ]:
# use train test split to get train and validation set
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled.text,train_df_shuffled.target, test_size=0.1,random_state=711)

In [ ]:
# check the length of the train data and labels
# should have equal length
len(train_sentences), len(train_labels)

(6851, 6851)

In [ ]:
# check the length of the validation data and labels
# should have equal length
len(val_sentences), len(val_labels)

(762, 762)

In [1]:
import pandas as pd
import numpy as np

In [27]:
df2 = pd.DataFrame(np.array([[1, 2, np.nan], [4, 5, 6], [7, 8, 9]]),
                   columns=['a', 'b', 'c'])
df2

,a,b,c
0,1.0,2.0,NaN
1,4.0,5.0,6.0
2,7.0,8.0,9.0


In [28]:
df2 = df2.fillna('1 2')

In [29]:
df2

,a,b,c
0,1.0,2.0,1 2
1,4.0,5.0,6
2,7.0,8.0,9


In [30]:
df2['c'].astype('str').str.replace(' ','')

0     12
1    6.0
2    9.0
Name: c, dtype: object

In [31]:
try: 
  df2.c.astype('int')
except:
  df2.c = df2['c'].astype('str').str.replace(" ","").astype('float').astype('int')


In [32]:
df2

,a,b,c
0,1.0,2.0,12
1,4.0,5.0,6
2,7.0,8.0,9


In [ ]:
df.append(df2.sort_values(by='a'),ignore_index=True)

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9
